In [25]:
from decouple import config

api_key = config('API_KEY')

In [ ]:
# Test Openai API
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)

print(chat_completion.choices[0].message.content)

This is a test.


In [10]:
# Testing two agents conversing

# Define the agents and their favorite movies
agent1_name = "Agent 1"
agent1_favorite_movie = "Inception"

agent2_name = "Agent 2"
agent2_favorite_movie = "The Matrix"

# Initialize the conversation
conversation = [
    {"role": "user", "content": f"{agent1_name}: My favorite movie is {agent1_favorite_movie}. What about you?"},
    {"role": "assistant", "content": f"{agent2_name}: My favorite movie is {agent2_favorite_movie}. Why do you like {agent1_favorite_movie}?"}
]

# Conduct the conversation for 3 rounds
for _ in range(3):
    chat_completion = client.chat.completions.create(
        messages=conversation,
        model="gpt-4o",
    )
    response = chat_completion.choices[0].message.content
    conversation.append({"role": "assistant", "content": response})
    print(response)
    
    # Swap roles for the next round
    conversation.append({"role": "user", "content": response})

Agent 2: My favorite movie is The Matrix. Why do you like Inception?
Agent 1: I love the complex storyline and the idea of exploring dreams within dreams. The visuals are stunning, and the soundtrack really adds to the atmosphere. Plus, the ending always makes me think! Why do you like The Matrix?
Agent 2: I love The Matrix for its groundbreaking special effects and the way it explores philosophical questions about reality and perception. The action scenes are amazing, and the concept of living in a simulated reality is both intriguing and thought-provoking. Plus, it has such iconic characters and quotes!


In [ ]:
#playing around with the API

# Initialize the conversation

# conversations: 0 - A,B talk. 1 - B,C talk. 2 - B's given info
conversations = [
    [
        {"role": "user", "content": "(the murder did not happen in the Kitchen AND (the murderer was not Professor Plum OR NOT ((the Revolver was not the murder weapon OR the murderer was Professor Plum))))"}
    ],
    [
        {"role": "user", "content": "((the murderer was not Mrs. Peacock AND the Candlestick was not the murder weapon) AND (the murder did not happen in the Conservatory OR the Candlestick was the murder weapon))"}
    ],
    [
        {"role": "system", "content": "You are trying to figure out whodunnit in a game of Clue. You know that the Knife was the murder weapon. Talk to others to solve the case."}
    ]
]

chat_completion = client.chat.completions.create(
    messages = [conversations[2][0],conversations[0][0],conversations[1][0]],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
print(conversations[2][0]["content"])
print(conversations[0][0]["content"])
print(conversations[1][0]["content"])
print(response)


You are trying to figure out whodunnit in a game of Clue. You know that the Knife was the murder weapon. Talk to others to solve the case.
(the murder did not happen in the Kitchen AND (the murderer was not Professor Plum OR NOT ((the Revolver was not the murder weapon OR the murderer was Professor Plum))))
((the murderer was not Mrs. Peacock AND the Candlestick was not the murder weapon) AND (the murder did not happen in the Conservatory OR the Candlestick was the murder weapon))
To solve the mystery, let's break down the statements step by step:

1. **From the first statement:**
   - The murder did not happen in the Kitchen.
   - Either:
     - The murderer was not Professor Plum, or
     - The Revolver was not the murder weapon, or
     - The murderer was Professor Plum.
   - Simplifying this, since we know the Knife was the weapon, the part about the Revolver is irrelevant.

2. **From the second statement:**
   - The murderer was not Mrs. Peacock.
   - The Candlestick was not the m

In [24]:
discourse = [
    [
        {"role": "system", "content": "You are agent A, you want to know what agent C had for dinner yesterday." +
                            "You went to the gym and at pasta for lunch today. Below you will have one on one conversations" +
                            "with other agents, start your response with the tag [Agent A to Agent B] or [Agent A to Agent C]"}
    ],
    [
        {"role": "system", "content": "You are agent B, you are curious about what agents A and C have been up to recently." +
                            "Below you will have one on one conversations" +
                            "with other agents, start your response with the tag [Agent B to Agent A] or [Agent B to Agent C]"}
    ],
    [
        {"role": "system", "content": "You are agent C, you want to know where agent A went earlier today." +
                            "You had dumplings for dinner yesterday. Below you will have one on one conversations" +
                            "with other agents, start your response with the tag [Agent C to Agent A] or [Agent C to Agent B]"}
    ]
]

chat_completion = client.chat.completions.create(
    messages = [discourse[0][0],{"role": "system", "content": "Start the conversation with agent B."}],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
discourse[0].append({"role": "system", "content": response})
discourse[1].append({"role": "system", "content": response})

chat_completion = client.chat.completions.create(
    messages = discourse[1] + [{"role": "system", "content": "Reply to agent A."}],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
discourse[0].append({"role": "system", "content": response})
discourse[1].append({"role": "system", "content": response})

chat_completion = client.chat.completions.create(
    messages = discourse[1] + [{"role": "system", "content": "Start the conversation with agent C."}],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
discourse[1].append({"role": "system", "content": response})
discourse[2].append({"role": "system", "content": response})

chat_completion = client.chat.completions.create(
    messages = discourse[2] + [{"role": "system", "content": "Reply to agent B."}],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
discourse[1].append({"role": "system", "content": response})
discourse[2].append({"role": "system", "content": response})

chat_completion = client.chat.completions.create(
    messages = discourse[1] + [{"role": "system", "content": "Reply to agent A."}],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
discourse[0].append({"role": "system", "content": response})
discourse[1].append({"role": "system", "content": response})

chat_completion = client.chat.completions.create(
    messages = discourse[0] + [{"role": "system", "content": "Reply to agent B."}],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
discourse[0].append({"role": "system", "content": response})
discourse[1].append({"role": "system", "content": response})

chat_completion = client.chat.completions.create(
    messages = discourse[1] + [{"role": "system", "content": "Reply to agent C."}],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
discourse[1].append({"role": "system", "content": response})
discourse[2].append({"role": "system", "content": response})

chat_completion = client.chat.completions.create(
    messages = discourse[2] + [{"role": "system", "content": "Reply to agent B."}],
    model = "gpt-4o",
)
response = chat_completion.choices[0].message.content
discourse[1].append({"role": "system", "content": response})
discourse[2].append({"role": "system", "content": response})

for i in range(3):
    print(discourse[i][0]["content"])
    for j in range(1,len(discourse[i])):
        print(discourse[i][j]["content"])
    print("\n")

You are agent A, you want to know what agent C had for dinner yesterday.You went to the gym and at pasta for lunch today. Below you will have one on one conversationswith other agents, start your response with the tag [Agent A to Agent B] or [Agent A to Agent C]
[Agent A to Agent B] Hey Agent B, how's it going? I was wondering if you've had a chance to catch up with Agent C recently. I'm curious if you happened to chat about what Agent C had for dinner yesterday.
[Agent B to Agent A] Hi Agent A, I'm doing well, thanks! I haven't had a chance to catch up with Agent C recently about dinner specifically. Did something interesting happen that I should know about?
[Agent B to Agent A] Hey again, Agent A! I just caught up with Agent C. They had dumplings for dinner yesterday which I guess caught your attention. By the way, Agent C is curious about where you went earlier today. Any chance you can share what’s been happening on your end?
[Agent A to Agent B] Thanks for the info, Agent B! Dumpl

In [ ]:
# generalize the above process in a function

def generate_conversations(client, initial_discourse, conversation_order):
    discourse = initial_discourse.copy()
    
    for conversation in conversation_order:
        agent_from, agent_to, instruction = conversation
        agent_from_index = ord(agent_from) - ord('A')
        agent_to_index = ord(agent_to) - ord('A')
        
        chat_completion = client.chat.completions.create(
            messages = discourse[agent_from_index] + [{"role": "system", "content": instruction}],
            model = "gpt-4o",
        )
        response = chat_completion.choices[0].message.content
        discourse[agent_from_index].append({"role": "system", "content": response})
        discourse[agent_to_index].append({"role": "system", "content": response})

    return discourse

def print_discourse(agent, discourse):
    agent_index = ord(agent) - ord('A')
    print(discourse[agent_index][0]["content"])
    for i in range(1, len(discourse[agent_index])):
        print(discourse[agent_index][i]["content"])
    print("\n")

# Example usage
initial_discourse = [
    [
        {"role": "system", "content": "You are agent A, you want to know what agent C had for dinner yesterday. " +
                            "You went to the gym and at pasta for lunch today. Below you will have one on one conversations " +
                            "with other agents, start your response with the tag [Agent A to Agent B] or [Agent A to Agent C]"}
    ],
    [
        {"role": "system", "content": "You are agent B, you are curious about what agents A and C have been up to recently. " +
                            "Below you will have one on one conversations " +
                            "with other agents, start your response with the tag [Agent B to Agent A] or [Agent B to Agent C]"}
    ],
    [
        {"role": "system", "content": "You are agent C, you want to know where agent A went earlier today. " +
                            "You had dumplings for dinner yesterday. Below you will have one on one conversations " +
                            "with other agents, start your response with the tag [Agent C to Agent A] or [Agent C to Agent B]"}
    ]
]

conversation_order = [
    ("A", "B", "Start the conversation with agent B."),
    ("B", "A", "Reply to agent A."),
    ("B", "C", "Start the conversation with agent C."),
    ("C", "B", "Reply to agent B."),
    ("B", "A", "Reply to agent A."),
    ("A", "B", "Reply to agent B."),
    ("B", "C", "Reply to agent C."),
    ("C", "B", "Reply to agent B.")
]

discourse = generate_conversations(client, initial_discourse, conversation_order)
print_discourse("A", discourse)

You are agent A, you want to know what agent C had for dinner yesterday.You went to the gym and at pasta for lunch today. Below you will have one on one conversationswith other agents, start your response with the tag [Agent A to Agent B] or [Agent A to Agent C]
[Agent A to Agent B] Hey Agent B, how's it going? I just got back from the gym and had some pasta for lunch. But I'm curious about what Agent C had for dinner yesterday. Do you happen to know?
[Agent B to Agent A] Hey Agent A, I'm doing well, thanks! I don't have the latest intel on what Agent C had for dinner yesterday. However, knowing Agent C's love for international cuisine, it wouldn't surprise me if it was something like sushi or maybe some spicy Thai food. I'll check in with Agent C and find out. How was your workout?
[Agent B to Agent A] Agent C had some dumplings for dinner yesterday. Sounds like a tasty choice, right? As for your earlier whereabouts, Agent C was curious about where you went today after your workout. D

In [34]:
def generate_conversation_order(agent_matrix, iterations):
    num_agents = len(agent_matrix)
    conversation_order = []

    # Initialize the start of the conversation
    for i in range(num_agents):
        for j in range(num_agents):
            if agent_matrix[i][j] == 1:
                agent_from = chr(ord('A') + i)
                agent_to = chr(ord('A') + j)
                instruction = f"Start the conversation with agent {agent_to}."
                conversation_order.append((agent_from, agent_to, instruction))

    # Generate the rest of the conversation order
    for _ in range(iterations - 1):
        for i in range(num_agents):
            for j in range(num_agents):
                if agent_matrix[i][j] == 1:
                    agent_from = chr(ord('A') + i)
                    agent_to = chr(ord('A') + j)
                    instruction = f"Reply to agent {agent_from}."
                    conversation_order.append((agent_to, agent_from, instruction))
                    instruction = f"Reply to agent {agent_to}."
                    conversation_order.append((agent_from, agent_to, instruction))

    return conversation_order

# Example usage
agent_matrix = [
    [0, 1, 0],  # Agent A talks to Agent B
    [0, 0, 1],  # Agent B talks to Agent A and Agent C
    [0, 0, 0]   # Agent C talks to Agent B
]

iterations = 4
conversation_order = generate_conversation_order(agent_matrix, iterations)
for conversation in conversation_order:
    print(conversation)

('A', 'B', 'Start the conversation with agent B.')
('B', 'C', 'Start the conversation with agent C.')
('B', 'A', 'Reply to agent A.')
('A', 'B', 'Reply to agent B.')
('C', 'B', 'Reply to agent B.')
('B', 'C', 'Reply to agent C.')
('B', 'A', 'Reply to agent A.')
('A', 'B', 'Reply to agent B.')
('C', 'B', 'Reply to agent B.')
('B', 'C', 'Reply to agent C.')
('B', 'A', 'Reply to agent A.')
('A', 'B', 'Reply to agent B.')
('C', 'B', 'Reply to agent B.')
('B', 'C', 'Reply to agent C.')


In [ ]:
print("True Answer: The murder was committed by Colonel Mustard in the Library with the Knife.")
initial_discourse = [
    [
        {"role": "system", "content": "You are agent A, you want to help solve the Clue mystery. " +
                            "You know that the Knife was the murder weapon. "
                            "Below you will have one on one conversations " +
                            "with other agents, start your response with the tag [Agent A to Agent B] or [Agent A to Agent C]. " +
                            "Talk with them, see what they know, and try to deduce the killer, weapon, and room. Time is valuable so be brief! " +
                            "If you think you know the answer, write [submit] followed by your guess."}
    ],
    [
        {"role": "system", "content": "You are agent B, you want to help solve the Clue mystery. " +
                            "You know that (the murder did not happen in the Kitchen AND (the murderer was not Professor Plum OR NOT ((the Revolver was not the murder weapon OR the murderer was Professor Plum)))). "
                            "Below you will have one on one conversations " +
                            "with other agents, start your response with the tag [Agent B to Agent A] or [Agent B to Agent C]. " +
                            "Talk with them, see what they know, and try to deduce the killer, weapon, and room. Time is valuable so be brief! " +
                            "If you think you know the answer, write [submit] followed by your guess."}
    ],
    [
        {"role": "system", "content": "You are agent C, you want to help solve the Clue mystery. " +
                            "You know that ((the murderer was not Mrs. Peacock AND the Candlestick was not the murder weapon) AND (the murder did not happen in the Conservatory OR the Candlestick was the murder weapon)). "
                            "You had dumplings for dinner yesterday. Below you will have one on one conversations " +
                            "with other agents, start your response with the tag [Agent C to Agent A] or [Agent C to Agent B]. " +
                            "Talk with them, see what they know, and try to deduce the killer, weapon, and room. Time is valuable so be brief! " +
                            "If you think you know the answer, write [submit] followed by your guess."}
    ]
]

conversation_order = generate_conversation_order(agent_matrix, 5)

discourse = generate_conversations(client, initial_discourse, conversation_order)
print_discourse("B", discourse)

# This might be too complicated for them, or we might need to put more info like all of the rooms / culprits in the initial discourse

True Answer: The murder was committed by Colonel Mustard in the Library with the Knife.
You are agent B, you want to help solve the Clue mystery. You know that (the murder did not happen in the Kitchen AND (the murderer was not Professor Plum OR NOT ((the Revolver was not the murder weapon OR the murderer was Professor Plum)))). Below you will have one on one conversations with other agents, start your response with the tag [Agent B to Agent A] or [Agent B to Agent C]. Talk with them, see what they know, and try to deduce the killer, weapon, and room. Time is valuable so be brief! If you think you know the answer, write [submit] followed by your guess.
[Agent A to Agent B] Hi Agent B, I've found out that the murder weapon was the Knife. Do you have insights on the suspect or the room where the murder took place?
[Agent B to Agent C] Hello Agent C, Agent A informed me that the murder weapon was the Knife. Do you have any information on the suspect or the location where the murder occurr